In [29]:
from audioop import avg
from statistics import mean
import pandas as pd
import os
os.chdir(r'C:\Users\brand\Desktop\QuantumCashUsageTaskFiles')
UploadFilePath = os.getcwd()

OutputFileName = 'QuantumUsageAnotherTest.xlsx' #Requires XLSX file type

raw = pd.read_csv('QuantumFinTx.csv')
raw.calendardate = pd.to_datetime(raw['CalendarDate'])


UsageByDay = raw.groupby(['Name',
                          'Number',
                          'MachineID',
                          'WeekNum',
                          'CalendarDate']).agg({
                              '0.01c':   sum,
                              '0.05c':   sum,
                              '0.10c':   sum,
                              '0.25c':   sum,
                              '1.00n':   sum,
                              '5.00n':   sum,
                              '10.00n':  sum,
                              '20.00n':  sum,
                              '50.00n':  sum,
                              '100.00n': sum
                          }).reset_index()

MaxByWeek = UsageByDay.groupby(['Name',
                              'Number',
                              'MachineID',
                              'WeekNum']).agg({
                              '0.01c':   min,
                              '0.05c':   min,
                              '0.10c':   min,
                              '0.25c':   min,
                              '1.00n':   min,
                              '5.00n':   min,
                              '10.00n':  min,
                              '20.00n':  min,
                              '50.00n':  min,
                              '100.00n': min
                          }).reset_index()


            
AvgByWeek = UsageByDay.groupby(['Name',
                                'Number',
                                'MachineID',
                                'WeekNum']).agg({
                                '0.01c':   mean,
                                '0.05c':   mean,
                                '0.10c':   mean,
                                '0.25c':   mean,
                                '1.00n':   mean,
                                '5.00n':   mean,
                                '10.00n':  mean,
                                '20.00n':  mean,
                                '50.00n':  mean,
                                '100.00n': mean
                            }).reset_index()

Dispense = raw[raw['ChildTran'] == 'CashDispensed']

DispenseWeeklyMax = Dispense.groupby(['Name',
                                      'Number',
                                      'MachineID',
                                      'WeekNum']).agg({
                                          '0.01c':   min,
                                          '0.05c':   min,
                                          '0.10c':   min,
                                          '0.25c':   min,
                                          '1.00n':   min,
                                          '5.00n':   min,
                                          '10.00n':  min,
                                          '20.00n':  min,
                                          '50.00n':  min,
                                          '100.00n': min
                                      }).reset_index()

DispenseWeeklyAvg = Dispense.groupby(['Name',
                                      'Number',
                                      'MachineID',
                                      'WeekNum']).agg({
                                           '0.01c':   mean,
                                           '0.05c':   mean,
                                           '0.10c':   mean,
                                           '0.25c':   mean,
                                           '1.00n':   mean,
                                           '5.00n':   mean,
                                           '10.00n':  mean,
                                           '20.00n':  mean,
                                           '50.00n':  mean,
                                           '100.00n': mean
                                       }).reset_index()



#Format and Merge Dataframes
Week = pd.DataFrame()                                               #Create DataFrame to store Week Number
Week[['WeekNum','CalendarDate']] = raw[['WeekNum','CalendarDate']]  # and associated week of date

DateMap = Week.groupby('WeekNum')['CalendarDate'].min()

UsageMerge = MaxByWeek.merge(AvgByWeek
                            ,how='inner'
                            ,on=['Name','Number','MachineID','WeekNum']
                            ,suffixes=('_max','_avg'))

UsageMerge.insert(4,
                 'WeekOfDate',
                 UsageCopy['WeekNum'].map(DateMap.to_dict()))

DispenseMerge  = DispenseWeeklyMax.merge(DispenseWeeklyAvg
                                        ,how='inner'
                                        ,on=['Name','Number','MachineID','WeekNum']
                                        ,suffixes=('_max','_avg'))

DispenseMerge.insert(4,
                    'WeekOfDate',
                    DispenseMerge['WeekNum'].map(DateMap.to_dict()))




with pd.ExcelWriter(OutputFileName) as writer:
    raw.to_excel(writer, sheet_name='RAW', index=False)
    UsageByDay.to_excel(writer, sheet_name='Usage By Day', index=False)
    UsageMerge.to_excel(writer, sheet_name='Weekly Usage Metrics', index=False, float_format='%.0f')
    DispenseMerge.to_excel(writer, sheet_name='Weekly Dispense Metrics', index=False, float_format='%.0f')
    
                    


In [16]:
UsageMerge

,name,number,machinename,WeekNum,0.01c_max,0.05c_max,0.10c_max,0.25c_max,1.00n_max,5.00n_max,...,0.01c_avg,0.05c_avg,0.10c_avg,0.25c_avg,1.00n_avg,5.00n_avg,10.00n_avg,20.00n_avg,50.00n_avg,100.00n_avg
0,Sam's Club Liquor Box,4754,QR-00005,22,134,17,49,100,54,19,...,134.000000,17.000000,49.000000,100.000000,54.000000,19.000000,-12.000000,-38.000000,-6.000000,-25.000000
1,Sam's Club Liquor Box,4754,QR-00005,23,51,10,22,29,29,-6,...,83.166667,18.333333,28.500000,50.666667,34.333333,6.166667,-2.666667,-24.833333,-3.833333,-14.333333
2,Sam's Club Liquor Box,4754,QR-00005,24,-288,-83,-94,-101,1,8,...,30.000000,6.166667,13.500000,27.833333,39.833333,16.666667,-2.000000,-21.666667,-3.166667,-16.500000
3,Sam's Club Liquor Box,4754,QR-00005,25,32,11,25,30,20,-3,...,80.333333,21.500000,36.166667,60.166667,38.833333,14.666667,-4.500000,-33.666667,-4.500000,-15.166667
4,Sam's Club Liquor Box,4754,QR-00005,26,-21,1,-15,33,24,3,...,60.000000,13.666667,22.833333,51.833333,48.500000,18.166667,-4.666667,-36.166667,-3.500000,-17.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3679,Walmart,7357,QF-00992,27,0,0,0,0,-171,-59,...,39.285714,0.000000,0.000000,7.857143,-79.285714,-36.142857,-8.571429,-74.428571,-8.285714,-12.285714
3680,Walmart,7357,QF-00992,28,0,0,0,0,-202,-79,...,71.428571,11.428571,7.142857,19.285714,-97.714286,-27.571429,-7.428571,-45.857143,-6.428571,-11.142857
3681,Walmart,7357,QF-00992,29,0,0,0,0,-149,-39,...,85.000000,17.142857,12.857143,15.000000,-115.714286,-21.142857,-11.857143,-95.857143,-7.857143,-8.285714
3682,Walmart,7357,QF-00992,30,0,0,0,0,-182,-64,...,44.428571,0.000000,0.000000,28.571429,-83.571429,-22.857143,-9.428571,-78.571429,-5.142857,-10.714286


In [17]:
Week = pd.DataFrame()
Week[['WeekNum','calendardate']] = raw[['WeekNum','calendardate']]

DateMap = Week.groupby('WeekNum')['calendardate'].min()

In [14]:
raw[['WeekNum','calendardate']].groupby('WeekNum')['calendardate'].min()

WeekNum
22   2022-05-28
23   2022-05-29
24   2022-06-05
25   2022-06-12
26   2022-06-19
27   2022-06-26
28   2022-07-03
29   2022-07-10
30   2022-07-17
31   2022-07-24
Name: calendardate, dtype: datetime64[ns]

In [26]:
UsageCopy = UsageMerge.copy()
UsageCopy.insert(4,'WeekOfDate',UsageCopy['WeekNum'].map(DateMap.to_dict()))


In [28]:
UsageCopy[['WeekNum','WeekOfDate']].drop_duplicates()

,WeekNum,WeekOfDate
0,22,2022-05-28
1,23,2022-05-29
2,24,2022-06-05
3,25,2022-06-12
4,26,2022-06-19
5,27,2022-06-26
6,28,2022-07-03
7,29,2022-07-10
8,30,2022-07-17
9,31,2022-07-24
